In [25]:
!pip install pandas
import pandas as pd
import numpy as np
import io
data = pd.read_json('gyapu_new.json')


In [3]:
!pip install sklearn


In [40]:
import re
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
data['description']=data['description'].apply(lambda cw : remove_tags(cw))
data["name_and_description"] = data["name"] + " " + data["description"]

In [116]:
from sklearn.model_selection import train_test_split


In [117]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)


In [118]:
train_data

,name,description,image,category,sub_category,name_and_description
7842,"2KG Glass Jar, Air-Tight Storage Containers Fo...",\n\tSet Of 4\n\tCapacity - 2 ltr / 2000 ml / 2...,public/files/8EBDC63DC7A22B8-2KG Bakery Glass ...,TV & Home Appliances,Kitchen Storage & Accessories,"2KG Glass Jar, Air-Tight Storage Containers Fo..."
5268,Mamaearth Vitamin C Nourishing Cold Cream 200gm,\n\tDull and dry skin begone! Get glowing with...,public/files/A06AAD1FCB44E0C-MP000000011188379...,Beauty & Health,Skin Care,Mamaearth Vitamin C Nourishing Cold Cream 200g...
6800,Digital Luggage Scale For Traveler Suitcase Ha...,\n\tSTOP WASTING MONEY - Never pay unexpected ...,public/files/BEF29F47EBC115A-luggage scale-gya...,TV & Home Appliances,Small Kitchen Appliances,Digital Luggage Scale For Traveler Suitcase Ha...
7146,H₂O PureMist™ Slim Wireless Ultrasonic Cool Mi...,\n\tLighting: 7 night light colors to select\n...,public/files/E48F048C052AC52-Untitled-2.png,TV & Home Appliances,Small Kitchen Appliances,H₂O PureMist™ Slim Wireless Ultrasonic Cool Mi...
5304,KOJA SHINSIA VIEW Premium Snail Cream 30g,"Feature\nContains Snail Secretion Filtrate, Hi...",public/files/AF537424264C144-Premium Snail Cre...,Beauty & Health,Skin Care,KOJA SHINSIA VIEW Premium Snail Cream 30g Feat...
...,...,...,...,...,...,...
5734,Himalaya Herbals Nourishing Cream And Honey So...,What it does:\n\n\n\tA specially formulated so...,public/files/17889DAC5C3ED61-honey-gyapu.jpg,Beauty & Health,Bath & Body,Himalaya Herbals Nourishing Cream And Honey So...
5191,P-Vita Melasma Cream,\n\tDermatologist Tested.\n\tSuitable for both...,public/files/7DBBBC16492D58E-80b2bb388db28b597...,Beauty & Health,Skin Care,P-Vita Melasma Cream \n\tDermatologist Tested....
5390,Olnature&#39;s Butox Face Energizer,&nbsp;\n\nProdcut details Of&nbsp;Olnature&#39...,public/files/9ADB3E494CB26BD-1_6f50195a-292f-4...,Beauty & Health,Skin Care,Olnature&#39;s Butox Face Energizer &nbsp;\n\n...
860,Luniva Sweet Red Wine 750ml,\n\tVolume:&nbsp;750ML\n\tBrand:&nbsp;Luniva\n...,public/files/4440A2F5F6F6BDF-IMG_3066.png,G-Bar,Wine,Luniva Sweet Red Wine 750ml \n\tVolume:&nbsp;7...


In [119]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])


In [120]:
from sklearn import preprocessing
encoder = preprocessing.OrdinalEncoder()

In [121]:
encoder.fit(np.reshape(data.category.to_numpy(), (len(data.category), 1)))
Y = encoder.transform(np.reshape(train_data.category.to_numpy(), (len(train_data.category), 1)))

In [122]:
Y = Y.reshape(-1)

In [123]:
text_clf.fit(train_data.name_and_description.to_numpy(), Y)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [124]:
target = encoder.transform(np.reshape(test_data.category.to_numpy(), (len(test_data.category), 1)))

In [125]:
predicted = text_clf.predict(test_data.name_and_description)

In [126]:
from sklearn import metrics
print(metrics.classification_report(target, predicted,))

              precision    recall  f1-score   support

         0.0       0.84      0.93      0.88       335
         1.0       0.95      0.86      0.90       181
         2.0       0.88      0.94      0.91       235
         3.0       0.95      0.97      0.96       297
         4.0       0.00      0.00      0.00        12
         5.0       1.00      0.81      0.89        62
         6.0       1.00      0.65      0.79       147
         7.0       1.00      0.51      0.67       128
         8.0       0.79      0.98      0.87       402

    accuracy                           0.88      1799
   macro avg       0.82      0.74      0.76      1799
weighted avg       0.89      0.88      0.87      1799



/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

In [127]:
from joblib import dump
dump(text_clf, f'models/CategoryClassifier.joblib') 
dump(encoder, f'encoders/CategoryEncoder.joblib')

['encoders/CategoryEncoder.joblib']

In [114]:
category_data = data.query("category.str.startswith('Groceries')")

In [115]:
train_data, test_data = train_test_split(category_data, test_size=0.2, random_state=42, shuffle=True)

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

sub_category_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [78]:
from sklearn import preprocessing
sub_category_encoder = preprocessing.OrdinalEncoder()

In [79]:
sub_category_encoder.fit(np.reshape(category_data.sub_category.to_numpy(), (len(category_data.sub_category), 1)))


OrdinalEncoder()

In [80]:
Y = sub_category_encoder.transform(np.reshape(train_data.sub_category.to_numpy(), (len(train_data.sub_category), 1)))

In [81]:
Y = Y.reshape(-1)

In [84]:
sub_category_clf.fit(train_data.name_and_description.to_numpy(), Y)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [87]:
target = sub_category_encoder.transform(np.reshape(test_data.sub_category.to_numpy(), (len(test_data.sub_category), 1)))
predicted = sub_category_clf.predict(test_data.name_and_description)
from sklearn import metrics
print(metrics.classification_report(target, predicted,))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         5
         1.0       1.00      0.55      0.71        11
         3.0       1.00      0.55      0.71        22
         4.0       1.00      0.05      0.09        21
         5.0       0.34      1.00      0.51        41
         6.0       0.00      0.00      0.00         5
         7.0       1.00      0.13      0.24        15
         8.0       0.00      0.00      0.00         4
         9.0       0.00      0.00      0.00         2
        10.0       0.00      0.00      0.00        16

    accuracy                           0.44       142
   macro avg       0.43      0.23      0.22       142
weighted avg       0.58      0.44      0.35       142



/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

In [132]:
from joblib import dump
categories = list(set(data["category"].values))
for category in categories:
    print(category)
    category_data = data.query(f"category.str.startswith('{category}')")
    train_data, test_data = train_test_split(category_data, test_size=0.2, random_state=42, shuffle=True)
    sub_category_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultinomialNB()),
    ])

    sub_category_encoder = preprocessing.OrdinalEncoder()

    sub_category_encoder.fit(np.reshape(category_data.sub_category.to_numpy(), (len(category_data.sub_category), 1)))
    Y = sub_category_encoder.transform(np.reshape(train_data.sub_category.to_numpy(), (len(train_data.sub_category), 1)))
    Y = Y.reshape(-1)

    sub_category_clf.fit(train_data.name_and_description.to_numpy(), Y)
    target = sub_category_encoder.transform(np.reshape(test_data.sub_category.to_numpy(), (len(test_data.sub_category), 1)))
    predicted = sub_category_clf.predict(test_data.name_and_description)
    from sklearn import metrics
    print(metrics.classification_report(target, predicted,))
    dump(sub_category_clf, f'models/{category}.joblib') 
    dump(sub_category_encoder, f'encoders/{category}.joblib')
    print("______________________")

Electronic Accessories
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         4
         2.0       0.97      1.00      0.98       176
         3.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00         1

    accuracy                           0.97       182
   macro avg       0.24      0.25      0.25       182
weighted avg       0.94      0.97      0.95       182



/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

______________________
Beauty & Health


/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        23
         1.0       0.90      0.39      0.55        23
         2.0       0.00      0.00      0.00        14
         3.0       0.00      0.00      0.00         9
         4.0       0.73      0.41      0.52        39
         5.0       0.00      0.00      0.00         4
         6.0       1.00      0.50      0.67        24
         7.0       1.00      0.13      0.24        30
         8.0       1.00      0.19      0.31        27
         9.0       0.00      0.00      0.00         6
        10.0       0.00      0.00      0.00         5
        11.0       0.00      0.00      0.00         8
        12.0       0.44      1.00      0.61       127
        13.0       0.00      0.00      0.00         2

    accuracy                           0.51       341
   macro avg       0.36      0.19      0.21       341
weighted avg       0.55      0.51      0.42       341

______________________
Gr

...

/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

______________________
G-Bar
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         4
         1.0       0.00      0.00      0.00         2
         2.0       0.00      0.00      0.00         2
         4.0       0.00      0.00      0.00         1
         5.0       1.00      1.00      1.00        11
         6.0       0.00      0.00      0.00         3
         7.0       1.00      0.33      0.50         6
         8.0       0.79      0.95      0.86        20
         9.0       0.72      1.00      0.84        21

    accuracy                           0.81        70
   macro avg       0.50      0.48      0.47        70
weighted avg       0.74      0.81      0.76        70

______________________
Fashion


/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         2
         1.0       0.83      0.99      0.90       142
         2.0       1.00      0.58      0.74        24
         3.0       0.99      0.87      0.93       125
         4.0       0.00      0.00      0.00         1

    accuracy                           0.90       294
   macro avg       0.56      0.49      0.51       294
weighted avg       0.90      0.90      0.89       294

______________________
Electronics
              precision    recall  f1-score   support

         0.0       1.00      0.76      0.87        17
         1.0       0.60      0.58      0.59        67
         2.0       0.00      0.00      0.00         1
         3.0       0.00      0.00      0.00         3
         4.0       0.64      0.87      0.74        85
         6.0       0.00      0.00      0.00        15
         8.0       1.00      0.20      0.33         5
         9.0       1.00      0.94      0.97

/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/suraj/projects/major_project/ecommerce_classifier/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

In [128]:
from joblib import load
text_clf = load("models/CategoryClassifier.joblib")
encoder = load("encoders/CategoryEncoder.joblib")

In [129]:
categories = encoder.inverse_transform([text_clf.predict(["healthy skin cream"])])

In [130]:
categories

array([['Beauty & Health']], dtype=object)